# Agentic RAG

문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들기

질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변 할 수 있음.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 **Agentic RAG** 라고 부른다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있음.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있음. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공함.


### 웹 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

`.env` 파일에 다음과 같이 등록합니다.

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("05-Agentic-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
05-Agentic-RAG


In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("i-ESG 기업의 전화번호는 무엇인가요?")

[{'url': 'https://www.bjfez.go.kr/kor/01866/01856/01808.web',
  'content': '참고 : ESG관련 온라인 세미나 자료 코참넷 홈페이지 바로가기. 담당부서: 기업정책과. 전화번호: 051-979-5322. 만족도 조사. 페이지의 내용이나 사용편의성에 만족'},
 {'url': 'https://www.facebook.com/people/i-ESG/100089226309495/',
  'content': 'i-ESG는 ESG 대응의 시급성과 중요성을 나누기 위해 지속 노력하고 있습니다. 인공지능 기반 ESG 특화 디지털 솔루션 기업으로서 기업들이 처한 현실적인 한계를 보다 쉽고'},
 {'url': 'https://i-esg.io/',
  'content': '기업들의 ESG 대응한계를 해소하는 ESG 특화 B2B 디지털 관리 솔루션. ... i-ESG는 막연했던 ESG 관리를 보다 편리하고 쉽게 관리할 수 있도록 돕습니다. ESG'},
 {'url': 'https://kr.linkedin.com/company/i-esg?trk=public_jobs_topcard_logo',
  'content': '웹사이트: https://i-esg.io. i-ESG 외부 링크 ; 업계: 정보 서비스 ; 회사 규모: 직원 2-10명 ; 유형: 비상장기업 ; 설립: 2022.'},
 {'url': 'https://i-esg.io/about',
  'content': '기업들의 ESG 대응한계를 해소하는 ESG 특화 B2B 디지털 관리 솔루션.'}]

### 문서 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.

**실습에 활용한 문서**

삼성전기 2023년 지속가능 보고서

- 파일명: `2023-2024_삼성전기_지속가능경영보고서.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 AzureOpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `FAISS` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [5]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/2023-2024_삼성전기_지속가능경영보고서.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(
    split_docs,
    AzureOpenAIEmbeddings(
        azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"],
        azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_EMBEDDINGS_API_KEY"],
    ),
)

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [6]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke(
    "2023년 삼성전기 지속가능경영보고서 중 지속가능경영 활동 내용을 찾아줘"
)

[Document(id='cb88bbd5-4247-4bb6-b42b-ae4bfd3ba682', metadata={'source': 'data/2023-2024_삼성전기_지속가능경영보고서.pdf', 'page': 0}, page_content='·\t에너지경영시스템 기반 \n전사 기후변화 리스크 \n통합 관리\n ·\t전사 기후전략위원회 \n운영(분기별 1회)\n ·\t주요 기후변화 리스크 \n관련 ESG위원회 보고를 \n통해 리스크 관리사후조치ESRS E1.IRO-1-20 (a) ESRS E1-4-32, ESRS E1-4-33\n자원순환 목표\n삼성전기는 2025년까지 전 사업장 폐기물 매립제로 인증 취득을 목표로 폐기물 유형 및 발생 원별 현황 추적을 통해 \n배출량을 관리하고, 폐기물 재활용 인프라 투자를 확대하여 재활용률을 95% 이상으로 달성할 계획입니다. 또한 \n2030년까지 용수 재이용률 36.7% 달성을 목표로 용수 재이용 시설 확대 및 수처리 설비 개선에 투자하는 등 용수 \n저감을 위해 적극적으로 활동하고 있습니다. ESRS E5-2-20 (b)Scope 3 온실가스 배출량 공개\n삼성전기는 밸류체인 전반 온실가스 배출량 관리를 위해 기업의 사업 전 과정에서 발생하는 Scope 3 온실가스 배출량 \n데이터를 투명하게 공개하고 있습니다. 또한 제3자 검증을 통하여 데이터 신뢰성을 향상하기 위해 노력하고 있습니다. GRI 305-3\nSUSTAINABLE BUSINESS AMBITION SUSTAINABLE PLANET SUSTAINABLE PROGRESS APPENDIX INTRODUCTION SUSTAINABLE PEOPLE \n환경영향 최소화 관리 제품환경 책임주의31\nTCFD Report'),
 Document(id='92d882c8-7376-4cb1-84b8-87098fd2bb03', metadata={'source': 'data/2023-2024_삼성전기_지속가능경영보고서.pdf', 'page': 3}, page_content='이

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환합니다.

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다. 

In [8]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## Agent 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. 

먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

**참고**
- 멀티턴 대화를 지원하지 않는다면 "chat_history" 를 제거해도 좋습니다.

In [10]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = AzureChatOpenAI(deployment_name="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

다음으로는 Tool Calling Agent 를 생성합니다.

In [11]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성합니다.

**참고**

- `verbose=False` 로 설정하여 중간 단계 출력을 생략하였습니다.

In [13]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.


In [18]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.invoke(
    {
        "input": "2023년 삼성전기 지속가능경영보고서 중 지속가능경영 활동 내용을 찾아서 어떤 활동인지 설명해줘"
    }
)

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '지속가능경영 활동'}`


지역 특성에 맞는 생물다양성 보전 활동을 시행한다.
이해관계자·지역사회와 지속적으로 소통하고 생물 다양성 보전에 공헌한다.
산업계의 지속가능한 생물자원 이용에 협력한다.
의사결정 시 생물다양성 보전을 고려한다.
생물다양성 공동선언문 6대 원칙
1생물다양성의 가치를 인식한다.
2정부는 생물다양성 보전 정책을 확대하고, 기업은 의사결정 시 생물다양성 보전을 고려한다.
3생물다양성 보전활동을 실천하기 위하여 함께 노력한다.
4산업계의 지속가능한 생물자원 이용을 위하여 협력한다.
5생물다양성 보전을 위하여 관련 국내외 기관과 교류하고 협력한다.
6생물다양성에 대한 사회 전반의 인식을 높이기 위해 노력한다.
SUSTAINABLE BUSINESS AMBITION SUSTAINABLE PLANET SUSTAINABLE PROGRESS APPENDIX INTRODUCTION SUSTAINABLE PEOPLE 
TCFD Report 제품환경 책임주의39
환경영향 최소화 관리

E4.IRO-1-17 (a), E4-3-27, E4-5-35
SUSTAINABLE BUSINESS AMBITION SUSTAINABLE PLANET SUSTAINABLE PROGRESS APPENDIX INTRODUCTION SUSTAINABLE PEOPLE 
TCFD Report 제품환경 책임주의40
환경영향 최소화 관리

이행한다.
 ·	목표 달성을 위한 리더십과 임직원의 적극적 
참여를 보장하고 방침을 이해관계자에게 공개하여 
투명성을 확보한다.환경, 에너지 경영 실천
 ·	제품 개발, 생산 등 전 과정에서 환경친화적인 
방법을 사용하여 환경보호에 앞장선다.
 ·	생산과정에서 발생하는 폐수, 폐기물 발생을 
최소화하고 자원과 에너지를 효율적으로 
사용하여 오염물질, 온실가스 배출을 줄이도록 
노력한다.안전하고 건강한 사업장 구축


`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [19]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.invoke(
    {"input": "삼성전기의 2023년 온실가스 총 배출량을 알려주세요"}
)

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '삼성전기 2023년 온실가스 총 배출량'}`


[{'url': 'https://kind.krx.co.kr/external/2024/06/28/000289/20240627000783/2023-2024%20%EC%82%BC%EC%84%B1%EC%A0%84%EA%B8%B0%20%EC%A7%80%EC%86%8D%EA%B0%80%EB%8A%A5%EA%B2%BD%EC%98%81%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf', 'content': '2023. ESRS. Scope 1, 2, 3. 배출량. 총 온실가스 배출량(Scope 1, 2, 3). (지역 기반). tCO2e. -. 1,479,213 1,354,578 1,655,830 1,624,979 1,984,830.'}, {'url': 'https://www.samsung.com/sec/sustainability/media/pdf/Samsung_Electronics_Sustainability_Report_2024_KOR.pdf', 'content': '삼성전자는 2023년 총 234억 원을 삼성 솔브포투모로우에. 지원했습니다. 1 ... 삼성전자는 온실가스 배출량을 Scope 1, 2, 3 로 구분하여 산출하고 지속가능경영'}, {'url': 'https://www.samsungena.com/filesview/Interactive_2023%20SAMSUNG%20E%26A%20Sustainability%20Report%20%28KOR%29_f.pdf?folder=%2Fupload%2Fpublishing%2F&storedfile=0fda8a54bcb048abab74d76072b14888.pdf', 'content': '삼성이앤에이 주식회사의 온실가스 보고서(2023년도) 관련. 검증 ... • 2023년도의 온실가스 배출 데이터 및 기

## 이전 대화내용 기억하는 Agent

이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
    verbose=True,
)

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.invoke(
    {
        "input": "2023년 삼성전기 지속가능경영보고서 중 지속가능경영 활동 내용을 문서에서 찾아서 어떤 활동인지 설명해줘"
    },
    # session_id 설정
    config={"configurable": {"session_id": "qwer"}},
)

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '2023 삼성전기 지속가능경영 활동'}`


·	에너지경영시스템 기반 
전사 기후변화 리스크 
통합 관리
 ·	전사 기후전략위원회 
운영(분기별 1회)
 ·	주요 기후변화 리스크 
관련 ESG위원회 보고를 
통해 리스크 관리사후조치ESRS E1.IRO-1-20 (a) ESRS E1-4-32, ESRS E1-4-33
자원순환 목표
삼성전기는 2025년까지 전 사업장 폐기물 매립제로 인증 취득을 목표로 폐기물 유형 및 발생 원별 현황 추적을 통해 
배출량을 관리하고, 폐기물 재활용 인프라 투자를 확대하여 재활용률을 95% 이상으로 달성할 계획입니다. 또한 
2030년까지 용수 재이용률 36.7% 달성을 목표로 용수 재이용 시설 확대 및 수처리 설비 개선에 투자하는 등 용수 
저감을 위해 적극적으로 활동하고 있습니다. ESRS E5-2-20 (b)Scope 3 온실가스 배출량 공개
삼성전기는 밸류체인 전반 온실가스 배출량 관리를 위해 기업의 사업 전 과정에서 발생하는 Scope 3 온실가스 배출량 
데이터를 투명하게 공개하고 있습니다. 또한 제3자 검증을 통하여 데이터 신뢰성을 향상하기 위해 노력하고 있습니다. GRI 305-3
SUSTAINABLE BUSINESS AMBITION SUSTAINABLE PLANET SUSTAINABLE PROGRESS APPENDIX INTRODUCTION SUSTAINABLE PEOPLE 
환경영향 최소화 관리 제품환경 책임주의31
TCFD Report

이행한다.
 ·	목표 달성을 위한 리더십과 임직원의 적극적 
참여를 보장하고 방침을 이해관계자에게 공개하여 
투명성을 확보한다.환경, 에너지 경영 실천
 ·	제품 개발, 생산 등 전 과정에서 환경친화적인 
방법을 사용하여 환경보호에 앞장선다.
 ·	생산과정에서 발생하는 폐수, 폐기물 발생을 
최소화하고 자원과 에너지를 효율적으로 
사용하

In [26]:
response = agent_with_chat_history.invoke(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "qwer"}},
)



> Entering new AgentExecutor chain...
Here is the translation of the previous response into English:

The sustainable management activities mentioned in Samsung Electro-Mechanics' 2023 Sustainability Management Report are as follows:

1. **Energy Management and Climate Change Response**:
   - Samsung Electro-Mechanics integrates the management of climate change risks across the company and operates a Climate Strategy Committee to manage these risks.
   - In 2022, the company joined RE100, aiming for a 100% transition to renewable energy by 2050, and in 2023, it secured 139,600 MWh of renewable energy.

2. **Resource Circulation Goals**:
   - The company aims to achieve zero waste to landfill certification at all business sites by 2025 and is working to achieve a recycling rate of over 95%.
   - By 2030, it aims to achieve a water reuse rate of 36.7% by expanding water reuse facilities.

3. **Greenhouse Gas Emission Management**:
   - The company transparently discloses Scope 3 greenh

## Agent 템플릿

다음은 전체 템플릿 코드 입니다.


In [36]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/2023-2024_삼성전기_지속가능경영보고서.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(
    split_docs,
    AzureOpenAIEmbeddings(
        azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"],
        azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_EMBEDDINGS_API_KEY"],
    ),
)

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = AzureChatOpenAI(deployment_name="gpt-4o-mini", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [38]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {"input": "구글이 앤스로픽에 투자한 금액을 웹에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '구글 앤스로픽 투자 금액'}`


[{'url': 'https://biz.chosun.com/it-science/ict/2023/10/29/SKHGD3YOKJEZNMNTC6NLMLUF6I/', 'content': '27일(현지시각) 월스트리트저널(WSJ)에 따르면 구글은 앤스로픽에 5억달러를 우선 투자했으며, 이후 15억달러를 추가 투자하기로 합의한 것으로 알려졌다.'}, {'url': 'https://www.etoday.co.kr/news/view/2440063', 'content': '구글, 오픈AI 대항마 ‘앤스로픽’에 10억 달러 추가 투자 - 이투데이 구글, 오픈AI 대항마 ‘앤스로픽’에 10억 달러 추가 투자 알파벳의 자회사인 구글이 인공지능(AI) 서비스 ‘클로드’ 개발사이자 오픈AI의 대항마로 꼽히는 앤스로픽에 10억 달러(약 1조4000억 원)를 추자 투자했다고 소식통을 인용해 파이낸셜타임스(FT)가 22일(현지시간) 보도했다. 블룸버그통신은 몇 주 전에 앤스로픽의 기업가치 600억 달러(약 87조 원)를 목표로 미국 벤처캐피털 기업 라이트스피드벤처파트너스 주도로 20억 달러의 자금 조달이 진행되고 있다고 전하기도 했다. 아마존은 지난해 11월 앤스로픽에 40억 달러를 투자하는 등 2023년 이후 지금까지 투자 규모만 80억 달러에 이른다. 이들 3개 기업은 합작회사를 통해 미국의 AI 산업에 최소 5000억 달러(718조 원)를 투자할 계획이다. [특징주] 케이씨에스, 양자 엑스퀀텀 참여 IDQ 국내 최초 국가인증 소식에 상승세 구글, 오픈AI 대항마 ‘앤스로픽’에 10억 달러 추가 투자 [아시아증시] 트럼프 AI 투자 소식에 일본 기술주 반색…닛케이 1.6%↑'}, {'url': 'https://www.hankyung.com/article/2023102958061', '

In [39]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...
Google has invested a total of $2 billion in Anthropic. This includes an initial investment of $500 million, with an agreement to invest an additional $1.5 billion later. (Sources: [Chosun Ilbo](https://biz.chosun.com/it-science/ict/2023/10/29/SKHGD3YOKJEZNMNTC6NLMLUF6I/), [EToday](https://www.etoday.co.kr/news/view/2440063), [Reuters](https://www.hankyung.com/article/2023102958061))

> Finished chain.
Google has invested a total of $2 billion in Anthropic. This includes an initial investment of $500 million, with an agreement to invest an additional $1.5 billion later. (Sources: [Chosun Ilbo](https://biz.chosun.com/it-science/ict/2023/10/29/SKHGD3YOKJEZNMNTC6NLMLUF6I/), [EToday](https://www.etoday.co.kr/news/view/2440063), [Reuters](https://www.hankyung.com/article/2023102958061))


In [40]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌'}, {'url': 'https://m.blog.naver.com/qhadldhaus98/223605904423', 'content': '2024 KBO 한국 프로야구 포스트시즌 일정 순위 예매 (와일드카드 준플레이오프 플레이오프 한국시리즈) : 네이버 블로그 2024 KBO 한국 프로야구 포스트시즌 일정 순위 예매 (와일드카드 준플레이오프 플레이오프 한국시리즈) 2024 KBO 한국 프로야구 포스트시즌 일정 순위 예매 (와일드카드 준플레이오프 플레이오프 한국시리즈) 1차전: 10월 5일 KT 3:2 LG 2차전: 10월 6일 KT 2:7 LG 3차전: 10월 8일 KT 5:6 LG 4차전: 10월 9일 KT 6:5 LG 5차전: 10월 11일 KT 1:4 LG 2024 KBO 프로야구 한국시리즈 일정 #보미블로그 #KBO #2024KBO #KBO리그 #포스트시즌 #가을야구 #와일드카드 #준플레이오프 #플레이오프 #한국시리즈 #KT위즈 #두산베어스 #LG트윈스 #삼성라이온즈 #기아타이거즈 #프로야구 #야구 #KBO포스트시즌 #가을야구 #프로야구와일드카드 #KBO준플레이오프 #KBO플레이오프 #프로야구한국시리즈 #프로야구포스트시즌 #야구랜선응원 #프로야구일정 #프로야구순위 #국내야구일정 #한국프로야구 #프로야구예매 #포스트시즌예매 {"title":"2024 KBO 한국 프로야구 포스트시즌 일정 순위 예

In [41]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

print(response['output'])



> Entering new AgentExecutor chain...
2024년 KBO 프로야구 플레이오프 진출팀은 KIA, 삼성, LG, KT, 두산입니다! 뜨거운 경쟁이 기대되네요! ⚾️🔥 #KBO #프로야구 #플레이오프

> Finished chain.
2024년 KBO 프로야구 플레이오프 진출팀은 KIA, 삼성, LG, KT, 두산입니다! 뜨거운 경쟁이 기대되네요! ⚾️🔥 #KBO #프로야구 #플레이오프


In [42]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024 KBO 한국 시리즈 일정'}`


[{'url': 'https://www.instagram.com/kbo.official/p/DBTTeamzOOX/', 'content': '2024 KBO 한국시리즈 일정 1차전 : 10월 21일(월) - 광주기아챔피언스필드 2차전 : 10월 22일(화) - 광주기아챔피언스필드 3차전 : 10월 24일(목) -'}, {'url': 'https://ko.wikipedia.org/wiki/2024%EB%85%84_%ED%95%9C%EA%B5%AD%EC%8B%9C%EB%A6%AC%EC%A6%88', 'content': '대진은 2024년 KBO 리그 정규 시즌 1위 팀으로 2017년 한국시리즈 이후 7년만에 한국시리즈 직행에 성공하며 이번 시즌 통합 우승이자 통산 12번째 우승에 도전하는 KIA 타이거즈와 정규시즌 2위이자 플레이오프 승리팀으로 2015년 한국시리즈 이후 9년만에 한국시리즈에 진출해 통산 9번째 우승에 도전하는 삼성 라이온즈가 맞붙었다. 10월 23일(수)  1차전 삼성 라이온즈 1 - 5   KIA 타이거즈    광주기아챔피언스필드  18시 30분 19,300명 (매진)    전상현 (KIA 타이거즈) 10월 23일(수)  2차전 삼성 라이온즈 3 - 8   KIA 타이거즈    19,300명 (매진)    양현종 (KIA 타이거즈) 10월 26일(토)  4차전 KIA 타이거즈    9 - 2   삼성 라이온즈 14시 00분 23,550명 (매진)    김태군 (KIA 타이거즈) KIA는 8회초 2사 만루 상황부터 등판했던 마무리 투수 정해영이 9회초 2사에서 마지막 타자 김성윤을 삼진 처리하며 경기를 마무리했고, 마침내 KIA 타이거즈가 시리즈 전적 4승 1패로 한국시리즈 불패 신화를 이어가며 7년 만의 통합 우승이자 팀의 12번째 